In [1]:
from datetime import datetime
from pathlib import Path
import pandas as pd
import numpy as np
import random as rd
import time,sys,os,sqlite3
import traceback
import backtrader as bt # 导入 Backtrader
import backtrader.indicators as btind # 导入策略分析模块
import backtrader.feeds as btfeeds # 导入数据模块

<figure class="half">
    <img src=./pic/bt_mind.png width="750">   
    <div>backtrader 模块导图 </div>
</figure>

通常回测流程如下  

**step 1：构建策略**

确定策略潜在的可调参数；  
计算策略中用于生成交易信号的指标；  
按需打印交易信息；  
编写买入、卖出的交易逻辑。

**step 2：实例化策略引擎 cerebro，由 cerebro 来驱动回测**|

由 DataFeeds 加载数据，再将加载的数据添加给 cerebro；  
将上一步生成的策略添加给 cerebro；  
按需添加策略分析指标或观测器；  
通过运行 cerebro.run() 来启动回测；  
回测完成后，按需运行 cerebro.plot() 进行回测结果可视化展示。

<figure class="half">
    <img src=./pic/bt_proc.png width="600">   
    <div>backtrader 流程导图 </div>
</figure>

In [ ]:
# 回测流程 代码流
# 创建策略
class TestStrategy(bt.Strategy):
    # 可选，设置回测的可变参数：如移动均线的周期
    params = (
        (...,...), # 最后一个逗号 最好别删！
    )
    def log(self, txt, dt=None):
        '''可选，构建策略打印日志的函数：可用于打印订单记录或交易记录等'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        '''必选，初始化属性、计算指标等'''
        pass

    def notify_order(self, order):
        '''可选，打印订单信息'''
        pass

    def notify_trade(self, trade):
        '''可选，打印交易信息'''
        pass

    def next(self):
        '''必选，编写交易策略逻辑'''
        sma = btind.SimpleMovingAverage(...) # 计算均线
        pass

# 实例化 cerebro
cerebro = bt.Cerebro()
# 通过 feeds 读取数据
data = btfeeds.BacktraderCSVData(...)
# 将数据传递给 “大脑”
cerebro.adddata(data)
# 通过经纪商设置初始资金
cerebro.broker.setcash(...)
# 设置单笔交易的数量
cerebro.addsizer(...)
# 设置交易佣金
cerebro.broker.setcommission(...)
# 添加策略
cerebro.addstrategy(TestStrategy)
# 添加策略分析指标
cerebro.addanalyzer(...)
# 添加观测器
cerebro.addobserver(...)
# 启动回测
cerebro.run()
# 可视化回测结果
cerebro.plot()


#### **case 1  空策略**

In [4]:
# 实例化 cerebro
cerebro = bt.Cerebro()
# 打印初始资金
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# 启动回测
cerebro.run()
# 打印回测完成后的资金
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10000.00


#### **case 2 回测，省略了选股过程，直接回测**
回测条件：  
pool：zz500  
interval：2019-01-01  2021-01-28  
period：每月第一个交易日，开盘价买入卖出  
weight：流通市值占比  
total：100000000  
cost：0.0003 both side  
滑点：0.0001 both side  
logic：每月最后一个交易日，基于选股规则，选出中证500中最优的20%，作为下个月的持仓

In [27]:
cerebro = bt.Cerebro()
# 数据导入
daily_price = pd.read_csv(Path("./bt_data/daily_price.csv"), index_col = 'datetime',  parse_dates=['datetime'])  #  daily_price.csv 对应的是 510 只股票各自从 2019-01-02 至 2021-01-28 的日度行情数据（后复权）
trade_info = pd.read_csv(Path("./bt_data/trade_info.csv"), index_col='trade_date',  parse_dates=['trade_date'])  # 最终的选股结果，共包含 3 个字段：trade_date 调仓期（每月最后一个交易日）、sec_code 持仓成分股代码、weight 持仓权重 。

# 按股票代码，依次循环传入数据
for stock_code in daily_price['sec_code'].unique():
    # 日期对齐
    data = pd.DataFrame(index=daily_price.index.unique()) # 获取回测区间内所有交易日
    df = daily_price.query(f"sec_code=='{stock_code}'")[['open','high','low','close','volume','openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    # 缺失值处理：日期对齐时会使得有些交易日的数据为空，所以需要对缺失数据进行填充
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(0)
    # 导入数据
    datafeed = bt.feeds.PandasData(dataname=data_, fromdate=datetime(2019,1,2), todate=datetime(2021,1,28))
    cerebro.adddata(datafeed, name=stock_code) # 通过 name 实现数据集与股票的一一对应
#     print("{} Done !".format(stock_code))
print("done")

done


**导入数据需要注意**  
1. 交易日不统一 上市，退市时间不统一，回测区间内停牌，所以需要填充交易日数据  
2. 行情确实 eg.将volume填充为0，表示股票无法交易，将缺失的高开低收做前向填充；将上市前缺失的高开低收填充为 0 等；  
3. 股票与行情数据的匹配 通过设置 adddata() 方法中 name 参数，来实现数据集与股票的一 一对应关系。

In [28]:
# 回测条件
# 初始资金 100,000,000
cerebro.broker.setcash(100000000.0)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)

# 通过 analyzers 策略分析模块和 observers 观测器模块提前配置好要返回的回测结果
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤

In [29]:
cerebro.analyzers

[(backtrader.analyzers.timereturn.TimeReturn, (), {'_name': 'pnl'}),
 (backtrader.analyzers.annualreturn.AnnualReturn,
  (),
  {'_name': '_AnnualReturn'}),
 (backtrader.analyzers.sharpe.SharpeRatio, (), {'_name': '_SharpeRatio'}),
 (backtrader.analyzers.drawdown.DrawDown, (), {'_name': '_DrawDown'})]

In [30]:
# 交易策略，基于调仓信息 trade_info.csv 在自定义策略中调仓
# 至少需要定义 __init__() 和 next() 方法。其中， __init__() 用于初始化各类属性，next() 用于下单交易

# 通过继承 Strategy 基类，来构建自己的交易策略子类
class MyStrategy(bt.Strategy):
    # 定义我们自己写的这个 MyStrategy 类的专有属性
    def __init__(self):
        '''必选，策略中各类指标的批量计算或是批量生成交易信号都可以写在这里'''
        self.buy_stock = trade_info # 保留调仓列表
        # 读取调仓日期，即每月的最后一个交易日，回测时，会在这一天下单，然后在下一个交易日，以开盘价买入
        self.trade_dates = pd.to_datetime(self.buy_stock.index.unique()).tolist()
        self.order_list = [] # 记录以往订单，方便调仓日对未完成订单做处理
        self.buy_stocks_pre = [] # 记录上一期持仓
    # 构建交易函数: 策略交易的主体部分
    def next(self):
        '''必选，在这里根据交易信号进行买卖下单操作'''
        dt = self.datas[0].datetime.date(0) # 获取当前的回测时间点
        # 如果是调仓日，则进行调仓操作
        if dt in self.trade_dates:
            print("--------------{} 为调仓日----------".format(dt))
            # 在调仓之前，取消之前所下的没成交也未到期的订单
            if len(self.order_list) > 0:
                for od in self.order_list:
                    self.cancel(od) # 如果订单未完成，则撤销订单
                self.order_list = [] #重置订单列表
            # 提取当前调仓日的持仓列表
            buy_stocks_data = self.buy_stock.query(f"trade_date=='{dt}'")
            long_list = buy_stocks_data['sec_code'].tolist()
            print('long_list', long_list) # 打印持仓列表
            # 对现有持仓中，调仓后不再继续持有的股票进行卖出平仓
            sell_stock = [i for i in self.buy_stocks_pre if i not in long_list]
            print('sell_stock', sell_stock) # 打印平仓列表
            if len(sell_stock) > 0:
                print("-----------对不再持有的股票进行平仓--------------")
                for stock in sell_stock:
                    data = self.getdatabyname(stock)
                    if self.getposition(data).size > 0 :
                        od = self.close(data=data)
                        self.order_list.append(od) # 记录卖出订单
            # 买入此次调仓的股票：多退少补原则
            print("-----------买入此次调仓期的股票--------------")
            for stock in long_list:
                w = buy_stocks_data.query(f"sec_code=='{stock}'")['weight'].iloc[0] # 提取持仓权重
                data = self.getdatabyname(stock)
                order = self.order_target_percent(data=data, target=w*0.95) # 为减少可用资金不足的情况，留 5% 的现金做备用
                self.order_list.append(order)
       
            self.buy_stocks_pre = long_list # 保存此次调仓的股票列表
        # 日志
    # 常用的有 notify_order() 订单日志、notify_trade() 交易日志、notify_cashvalue() 资金信息、notify_store() 交易事件说明等等
#     def notify_order(self, order):
#         # 未被处理的订单
#         if order.status in [order.Submitted, order.Accepted]:
#             return
#         # 已经处理的订单
#         if order.status in [order.Completed, order.Canceled, order.Margin]:
#             if order.isbuy():
#                 self.log(
#                         'BUY EXECUTED, ref:%.0f，Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
#                         (order.ref, # 订单编号
#                          order.executed.price, # 成交价
#                          order.executed.value, # 成交额
#                          order.executed.comm, # 佣金
#                          order.executed.size, # 成交量
#                          order.data._name)) # 股票名称
#             else: # Sell
#                 self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
#                             (order.ref,
#                              order.executed.price,
#                              order.executed.value,
#                              order.executed.comm,
#                              order.executed.size,
#                              order.data._name))
            
# 将编写的策略添加给大脑，别忘了 ！
cerebro.addstrategy(MyStrategy)

0

**some tips**  
1. Backtrader 默认情况下是：在 t 日运行下单函数，然后在  t+1 日以开盘价成交
2. 对于复杂的选股策略，建议参考本文的方式，事先确定好调仓日期、成分、权重，再将结果导入 Backtrader 做回测；
3. 一些函数说明：  
    self.close() 平仓  
    self.buy() 做多  
    self.sell() 做空  
    self.cancel() 取消订单  
    self.order_target_percent() 按持仓百分比下单，“多退少补”原则， 对于股票当前无持仓或持有的是多单（size>=0）的情况，若目标占比 target > 当前持仓占比，买入不够的部分；若目标占比 target < 当前持仓占比，卖出多余的部分。

In [31]:
# 启动回测
result = cerebro.run()
# 从返回的 result 中提取回测结果
strat = result[0]
# 返回日度收益率序列
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())
# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())

C:\Users\JD_Li\AppData\Local\Temp\ipykernel_23288\1560771575.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if dt in self.trade_dates:


--------------2019-01-31 为调仓日----------
long_list ['000006.SZ', '000008.SZ', '000025.SZ', '000090.SZ', '000536.SZ', '000587.SZ', '000598.SZ', '000612.SZ', '000636.SZ', '000656.SZ', '000690.SZ', '000712.SZ', '000766.SZ', '000807.SZ', '000829.SZ', '000877.SZ', '000980.SZ', '000999.SZ', '002002.SZ', '002048.SZ', '002051.SZ', '002074.SZ', '002110.SZ', '002127.SZ', '002128.SZ', '002131.SZ', '002152.SZ', '002195.SZ', '002308.SZ', '002358.SZ', '002359.SZ', '002375.SZ', '002400.SZ', '002408.SZ', '002437.SZ', '002463.SZ', '002465.SZ', '002642.SZ', '002707.SZ', '002745.SZ', '002818.SZ', '300001.SZ', '300010.SZ', '300058.SZ', '300113.SZ', '300146.SZ', '300166.SZ', '300266.SZ', '300376.SZ', '300450.SZ', '600006.SH', '600039.SH', '600053.SH', '600056.SH', '600062.SH', '600141.SH', '600151.SH', '600158.SH', '600169.SH', '600259.SH', '600260.SH', '600280.SH', '600366.SH', '600373.SH', '600392.SH', '600393.SH', '600428.SH', '600478.SH', '600500.SH', '600525.SH', '600528.SH', '600582.SH', '600598.SH', 

In [33]:
# 从返回的 result 中提取回测结果
strat = result[0]
# 返回日度收益率序列
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())
# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())

--------------- AnnualReturn -----------------
OrderedDict([(2019, 0.2421668400755459), (2020, 0.2154227563253983), (2021, 0.017567210073598405)])
--------------- SharpeRatio -----------------
OrderedDict([('sharperatio', 1.4813312115232609)])
--------------- DrawDown -----------------
AutoOrderedDict([('len', 136), ('drawdown', 6.655064560819013), ('moneydown', 10952970.349310696), ('max', AutoOrderedDict([('len', 206), ('drawdown', 20.374812759676267), ('moneydown', 27705182.493407518)]))])
